Folium map (draw on region boundaries, colour code different types of bike lanes) (as at current)

In [14]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [16]:
bike_lane_gdf = gpd.read_file('../Clean Data/bicycle_data.geojson')

display(bike_lane_gdf)

,_id,INSTALLED,STREET_NAME,INFRA_LOWORDER,INFRA_HIGHORDER,WARD NAME,WARD NUMBER,geometry
0,1,2001,Kilbarry Rd,Sharrows - Wayfinding,Sharrows - Wayfinding,Toronto-St. Paul's,12,"MULTILINESTRING ((628651.515 4839265.273, 6286..."
1,2,2001,Martin Goodman / Waterfront Trl,Multi-Use Trail,Multi-Use Trail,Spadina-Fort York,10,"MULTILINESTRING ((628769.688 4832571.759, 6287..."
2,3,2001,Gatineau Hydro Corridor Trl,Multi-Use Trail,Multi-Use Trail,Scarborough Centre,21,"MULTILINESTRING ((638882.149 4844617.068, 6388..."
3,4,2001,G Ross Lord Park Trl,Multi-Use Trail,Multi-Use Trail,York Centre,06,"MULTILINESTRING ((623319.222 4847699.258, 6233..."
4,5,2001,Etobicoke Creek Trl,Multi-Use Trail - Entrance,Multi-Use Trail - Entrance,Etobicoke-Lakeshore,03,"MULTILINESTRING ((617202.250 4827390.434, 6172..."
...,...,...,...,...,...,...,...,...
1607,1461,2023,Bloor St W,Cycle Track,Cycle Track,Etobicoke-Lakeshore,03,"MULTILINESTRING ((617962.801 4833079.532, 6179..."
1608,1461,2023,Bloor St W,Cycle Track,Cycle Track,Etobicoke Centre,02,"MULTILINESTRING ((617962.801 4833079.532, 6179..."
1609,1462,2023,Kipling Ave,Cycle Track,Cycle Track,Etobicoke-Lakeshore,03,"MULTILINESTRING ((618219.487 4832846.809, 6182..."
1610,1463,2023,Dundas St W,Cycle Track,Cycle Track,Etobicoke-Lakeshore,03,"MULTILINESTRING ((617959.577 4832823.933, 6179..."


In [18]:
bike_lane_gdf['INFRA_LOWORDER'].value_counts()

INFRA_LOWORDER
Multi-Use Trail                         351
Signed Route (No Pavement Markings)     211
Sharrows                                192
Multi-Use Trail - Entrance              179
Bike Lane                               168
                                        134
Cycle Track                             114
Sharrows - Wayfinding                    98
Multi-Use Trail - Boulevard              50
Park Road                                38
Sharrows - Arterial - Connector          17
Multi-Use Trail - Existing Connector     16
Bike Lane - Buffered                     14
Multi-Use Trail - Connector              10
--                                        9
Sharrows - Arterial                       6
<Null>                                    4
---                                       1
Name: count, dtype: int64

In [38]:
# Create a style function to assign different colours to different cycling infrastructure

##########################################################################################################################################################
def style_function(feature):
    # Access the trail type from the feature's properties
    bike_path_type = feature['properties']['INFRA_LOWORDER']  

    # Set color based on the trail type
    if bike_path_type == 'Signed Route (No Pavement Markings)':
        color = 'red'
    elif bike_path_type in ['Sharrows', 'Sharrows - Wayfinding', 'Sharrows - Arterial - Connector', 'Sharrows - Arterial']:
        color = 'orange'
    elif bike_path_type in ['Multi-Use Trail', 'Multi-Use Trail - Entrance', 'Multi-Use Trail - Boulevard', 'Multi-Use Trail - Existing Connector', 'Multi-Use Trail - Connector']:
        color = 'green'
    elif bike_path_type in ['Bike Lane', 'Cycle Track', 'Bike Lane - Buffered']:
        color = 'blue'
    else:
        color = 'gray'  # Default color for other trail types

    return {
        'fillColor': color,  # Fill color for polygons (or lines)
        'color': color,      # Border color for polygons (or lines)
        'weight': 2,         # Border thickness
        'fillOpacity': 0.5   # Transparency of the fill color (if a polygon)
    }

##########################################################################################################################################################

In [40]:
# Create a map of Toronto
bike_lane_network = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Overlay the bike lanes on to the map of Toronto
folium.GeoJson(
    bike_lane_gdf,  # The GeoDataFrame to plot
    style_function=style_function  # Apply the custom style function
).add_to(bike_lane_network)

# Display map
bike_lane_network